# Batch run models
1. Running model
2. Evaluating results

In [ ]:
%load_ext lab_black
import os, json, multiprocessing, random
import pandas as pd
import altair as alt
import papermill as pm
from time import sleep
from meta import model_cfg, make_batch_cfg, check_cfgs_params, parse_batch_results, send_mail
from evaluate import make_df_wnw

## Make configs

In [ ]:
batch_name = "O2P_m270_rep"
batch_output_dir = "batch_eval/{}/".format(batch_name)

# Gen OG random seed: [int(random.random() * 1e5) for x in range(10)]

param_grid = {
    "p_noise": [0.0, 1.0, 3.0],
    "hidden_units": [50, 100, 250],
    "learning_rate": [0.05, 0.01, 0.02],
    "rng_seed":
        [53797, 77501, 86017, 61301, 96533, 88467, 85362, 66721, 49596, 71461]
}

static_hpar = {
    "sample_name": "jay",
    "cleanup_units": 20,
    "use_semantic": False,
    "input_dim": 119,
    "output_dim": 250,
    "pretrain_attractor": False,
    "rnn_activation": "sigmoid",
    "regularizer_const": None,
    "w_initializer": "glorot_uniform",
    "tau": 0.2,
    "max_unit_time": 4.0,
    "output_ticks": 20,
    "optimizer": "adam",
    "n_mil_sample": 1.0,
    "batch_size": 128,
    "save_freq": 10,
    "bq_dataset": batch_name,
}

batch_cfgs = make_batch_cfg(
    batch_name, static_hpar, param_grid, "OSP_master.ipynb"
)

## Parallel run

In [ ]:
# Run
def run_batch(cfg):
    """
    Using papermill to run parameterized notebook
    """
    print("Running model {}".format(cfg['sn']))
    os.makedirs(cfg['model_folder'], exist_ok=True)
    pm.execute_notebook(
        cfg['in_notebook'], cfg['out_notebook'], parameters=cfg['params']
    )


# Run in parallel pool
with multiprocessing.Pool(4) as pool:
    pool.map(run_batch, batch_cfgs)

Compile and save results

In [ ]:
cfgs, df = parse_batch_results(batch_cfgs)
df.to_csv(batch_output_dir + 'bcdf.csv')
cfgs.to_csv(batch_output_dir + 'cfgs.csv')

Shutdown compute engine

In [ ]:
sleep(30)
print('shuting down')
!sudo poweroff

## Plotting

Review the batch structure

In [ ]:
cfgs = pd.read_csv(batch_output_dir + 'cfgs.csv')
df = pd.read_csv(batch_output_dir + 'bcdf.csv')

In [ ]:
cfgs.loc[cfgs.learning_rate == 0.05, ]

In [ ]:
check_cfgs_params(cfgs)

Create re-useable overview heatmap and word vs. nonword df

In [ ]:
variates = ['hidden_units', 'learning_rate', 'p_noise']

df = df.pivot_table(
    index=['hidden_units', 'learning_rate', 'p_noise', 'epoch', 'cond']
).reset_index()

In [ ]:
alt.data_transformers.enable("default")
alt.data_transformers.disable_max_rows()

# Selectors for interactions
sel_run = alt.selection(type="multi", on="click", fields=["code_name"])

########## Overview heatmap model selector ##########
df_ov = df[(df.epoch == df.epoch.max()) & (df.timestep == df.timestep.max())]

overview = (
    alt.Chart(df_ov).mark_rect().encode(
        x="p_noise:O",
        y="hidden_units:O",
        row="learning_rate:O",
        column="cleanup_units:O",
        color=alt.Color(
            "mean(acc):Q",
            scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1))
        ),
        opacity=alt.condition(sel_run, alt.value(1), alt.value(0.1)),
        tooltip=["acc"] + variates,
    ).add_selection(sel_run).properties(
        title="Overall accuracy (incl. all words and nonwords)"
    )
)

overview

In [ ]:
########## Word vs. Nonword accuracy over epoch ##########
df_wnw = make_df_wnw(
    df, selected_cond=['INC_HF', 'unambiguous']
)  # Match with Rueckl
df_wnw = df_wnw.merge(cfgs)

mdf_wnw = df_wnw.melt(
    id_vars=['code_name', 'epoch'],
    value_vars=['word_acc', 'nonword_acc'],
    var_name='wnw',
    value_name='acc'
)

plot_epoch = alt.Chart(mdf_wnw).mark_point(size=80).encode(
    y=alt.Y("mean(acc):Q", scale=alt.Scale(domain=(0, 1))),
    x="epoch:Q",
    color=alt.Color("code_name", legend=None),
    shape="wnw:N",
    opacity=alt.condition(sel_run, alt.value(1), alt.value(0)),
    tooltip=["code_name", "epoch", "acc"],
).add_selection(sel_run).properties(
    title="Plot word and nonword accuracy by epoch"
)

########## Word vs. Nonword plot with diagonal ##########

wnw_line = alt.Chart(df_wnw).mark_line().encode(
    y=alt.Y("mean(nonword_acc):Q", scale=alt.Scale(domain=(0, 1))),
    x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
    color=alt.Color("code_name", legend=None),
    opacity=alt.condition(sel_run, alt.value(1), alt.value(0)),
    tooltip=["code_name", "epoch", "word_acc", "nonword_acc"] + variates,
)

wnw_point = wnw_line.mark_point().encode(
    color=alt.Color("epoch", scale=alt.Scale(scheme="redyellowgreen")),
)

diagonal = alt.Chart(pd.DataFrame({
    'x': [0, 1],
    'y': [0, 1]
})).mark_line(color='black').encode(x='x', y='y')

wnw = diagonal + wnw_line + wnw_point

wnw_interactive = wnw.add_selection(sel_run).properties(
    title="Word vs. Nonword accuracy at final time step"
)

dashboard = overview | plot_epoch | wnw_interactive
dashboard.save(batch_output_dir + 'dashboard.html')
dashboard

Main effect plot

In [ ]:
def main_effect_plot(df, var):

    pdf = df.pivot_table(
        index=['epoch', var], values=['word_acc', 'nonword_acc']
    )

    pdf.reset_index(inplace=True)
    pdf['word_advantage'] = pdf.word_acc - pdf.nonword_acc

    sel_run = alt.selection(type="multi", on="click", fields=[var])

    overview_wacc = alt.Chart(pdf).mark_rect().encode(
        y=alt.Y(var, type="ordinal"),
        x='epoch:O',
        color=alt.Color(
            "word_acc", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1))
        ),
        opacity=alt.condition(sel_run, alt.value(1), alt.value(0.1)),
        tooltip=["word_acc", "nonword_acc"],
    ).add_selection(sel_run).properties(
        title="Word - Nonword accuracy (word accuracy) heatmap by {} and epoch".
        format(var)
    )

    overview_wadv = overview_wacc.encode(
        color=alt.Color(
            "word_advantage",
            scale=alt.Scale(scheme="redyellowgreen", domain=(-.2, .2))
        )
    ).properties(
        title="Word - Nonword accuracy (word advantage) heatmap by {} and epoch"
        .format(var)
    )

    wnw_line = alt.Chart(pdf).mark_line().encode(
        y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
        x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
        color=alt.Color(var, type="ordinal", scale=alt.Scale(scheme="magma")),
        opacity=alt.condition(sel_run, alt.value(0.9), alt.value(0)),
        tooltip=[var, "epoch", "word_acc", "nonword_acc"],
    )

    diagonal = alt.Chart(pd.DataFrame({
        'x': [0, 1],
        'y': [0, 1]
    })).mark_line(color='black').encode(x='x', y='y')

    return overview_wacc | overview_wadv | (diagonal + wnw_line)


p = alt.vconcat()
for v in variates:
    p &= main_effect_plot(df_wnw, v)

p.save(batch_output_dir + 'main_effects.html')
p

In [ ]:
df_wnw['word_advantage'] = df_wnw.word_acc - df_wnw.nonword_acc

In [ ]:
base = alt.Chart(df_wnw).mark_point().encode(
    y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
    x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
    color=alt.Color(
        "epoch", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 100))
    ),
    opacity=alt.value(0.2),
    tooltip=["code_name", "epoch", "word_acc", "nonword_acc"],
)

diagonal = alt.Chart(pd.DataFrame({
    'x': [0, 1],
    'y': [0, 1]
})).mark_line(color='black').encode(x='x', y='y')

plot = diagonal + base

plot.save(batch_output_dir + 'all_models_wnw.html')
plot

In [ ]:
base = alt.Chart(df_wnw).mark_rect().encode(
    x="p_noise:O",
    y="hidden_units:O",
    row="learning_rate:O",
    column="epoch:O",
    tooltip=[
        "p_noise", "hidden_units", "cleanup_units", "learning_rate", "word_acc",
        "nonword_acc"
    ],
)

w = base.encode(
    color=alt.
    Color("word_acc", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1)))
).properties(title="Word acc")

w.save(batch_output_dir + 'word.html')

In [ ]:
nw = base.encode(
    color=alt.Color(
        "nonword_acc", scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1))
    )
).properties(title="Nonword acc")
nw.save(batch_output_dir + 'nonword.html')

In [ ]:
adv = base.encode(
    color=alt.Color(
        "word_advantage",
        scale=alt.Scale(scheme="redyellowgreen", domain=(-.3, .3))
    )
).properties(title="Word advantage")

adv.save(batch_output_dir + 'wadv.html')

In [ ]:
df_agg = df_wnw.pivot_table(
    index=['epoch', 'learning_rate', 'hidden_units', 'p_noise']
)

df_agg.reset_index(inplace=True)
df_agg

In [ ]:
plot_pnoise = alt.Chart().mark_line().encode(
    y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
    x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
    color=alt.Color("p_noise", type="ordinal", scale=alt.Scale(scheme="reds")),
    tooltip=[
        "epoch", "hidden_units", "cleanup_units", "p_noise", "learning_rate",
        "word_acc", "nonword_acc"
    ],
)

diagonal = alt.Chart(pd.DataFrame({
    'x': [0, 1],
    'y': [0, 1]
})).mark_line(color='black').encode(x='x', y='y')

p = alt.layer(diagonal + plot_pnoise, data=df_agg).facet(
    row="hidden_units:O", column="learning_rate:O"
)

p.save(batch_output_dir + 'p_noise.html')

In [ ]:
plot_hidden = alt.Chart().mark_line().encode(
    y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
    x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
    color=alt.Color(
        "hidden_units", type="ordinal", scale=alt.Scale(scheme="blues")
    ),
    tooltip=[
        "epoch", "hidden_units", "cleanup_units", "p_noise", "learning_rate",
        "word_acc", "nonword_acc"
    ],
)

h = alt.layer(diagonal + plot_hidden,
              data=df_agg).facet(row="p_noise:O", column="learning_rate:O")

h.save(batch_output_dir + 'hidden.html')

In [ ]:
plot_lr = alt.Chart().mark_line().encode(
    y=alt.Y("nonword_acc:Q", scale=alt.Scale(domain=(0, 1))),
    x=alt.X("word_acc:Q", scale=alt.Scale(domain=(0, 1))),
    color=alt.Color(
        "learning_rate", type="ordinal", scale=alt.Scale(scheme="greens")
    ),
    tooltip=[
        "epoch", "hidden_units", "cleanup_units", "p_noise", "learning_rate",
        "word_acc", "nonword_acc"
    ],
)

lr = alt.layer(diagonal + plot_lr,
               data=df_agg).facet(row="hidden_units:O", column="p_noise:O")

lr.save(batch_output_dir + 'lr.html')

Save notebook to html

In [ ]:
!jupyter nbconvert --output-dir=$batch_output_dir --to html batch.ipynb